In [1]:
# Do we need to parallelize? Yes.

In [22]:
from __future__ import division
import numpy as np
import random
import multiprocessing as mp
from collections import Counter
import functools
import datetime
import matplotlib.pyplot as plt
import seaborn
import math
import datetime as dt
import networkx as nx
import operator

In [80]:
def loadEdgelist(filename):
	# Read in graph
	g = nx.read_edgelist(filename, 
	                     delimiter="\t", 
	                     create_using  = nx.DiGraph(), 
	                     data=True)

	# Convert Weight To Float
	temp = map(lambda (x, y): (x, float(y)), 
			   nx.get_edge_attributes(g, "weight").items())
	nx.set_edge_attributes(g, "weight", dict(temp))

	# Convert Date to datetime
	temp = map(lambda (x, y): (x, dt.datetime.strptime(y, "%Y-%m-%d")), 
			   nx.get_edge_attributes(g, "date").items())
	nx.set_edge_attributes(g, "date", dict(temp))

	return g

def writeEdgelist(g, filename):
    # Convert date to string
    temp = map(lambda (x, y): (x, y.strftime('%Y-%m-%d')), 
               nx.get_edge_attributes(g, "date").items())
    nx.set_edge_attributes(g, "date", dict(temp))
    
    # Write to file
    nx.write_edgelist(g,filename, delimiter="\t", data=True)

preG = loadEdgelist("../1_snapshotting/cumulative_snapshots/enddate_20140228.edgelist")
postG = loadEdgelist("../1_snapshotting/cumulative_snapshots/enddate_20150831.edgelist")

# Enforce Bipartiteness
# Set node attributes. 0 is an investor, 1 is a company.
node_attributes =dict(map(lambda (x,y): (x,1) if y > 0 else (x,0), 
                      preG.in_degree().items()))
nx.set_node_attributes(preG, "bipartite", node_attributes)

In [81]:
def create_truth(preG, postG):
    if postG postG.edges()

[(u'/organization/rsj-private-equity', u'/organization/cognitive-security'),
 (u'/organization/rsj-private-equity', u'/organization/beepl'),
 (u'/organization/petra-partners', u'/organization/new-century-hospice'),
 (u'/person/kevin-lin', u'/organization/the-ticket-fairy'),
 (u'/person/emilios-markou', u'/organization/park-around'),
 (u'/person/dave-leyrer', u'/organization/tapiture'),
 (u'/person/dave-leyrer', u'/organization/omaze'),
 (u'/person/oren-etzioni', u'/organization/algorithmia'),
 (u'/organization/beamonte-investments', u'/organization/arthena'),
 (u'/organization/beamonte-investments', u'/organization/black-house'),
 (u'/organization/beamonte-investments', u'/organization/master-kiwi'),
 (u'/organization/beamonte-investments', u'/organization/kiwii-capital'),
 (u'/organization/beamonte-investments', u'/organization/nanosatisfi'),
 (u'/organization/inventures', u'/organization/foruforever'),
 (u'/organization/inventures', u'/organization/rewarder'),
 (u'/organization/healt

In [66]:
# Template for what a scoring function should look like
# Input --> source and destination, reference to graph
# Output --> score (how do we normalize???)
# g is passed by reference, by default, in python
def test_score(investor, company, g):
    return 0.5

# return a random float as the score
def rand_score(investor, company, g):
    return random.random()

# calculate the preferential attachment score (normalized to the max preferential attachment score)
# max(preG.in_degree().values())*max(preG.out_degree().values()) ---> 29,000
def pa_score(investor, company, g):
    investor_degree = g.out_degree(investor)
    company_degree = g.in_degree(company)
    
    if investor_degree == 0: investor_degree = 1
    if company_degree == 0: company_degree = 1
    return investor_degree*company_degree/29000
    


In [67]:
preG.nodes()
len(preG[u'/organization/rsj-private-equity'])*len(preG[u'/organization/rsj-private-equity'])
pa_score(u'/organization/sequoia-capital', u'/organization/uber', preG)

0.35

In [78]:
# We want to factor our code so that it is efficient for simple multiprocessing
#
# Given a LIST of investors, a SINGLE company, calculate the number of TP/TN/FP/FN
# and return as a counter. This is meant to run inside of eval_prec_recall.
def company_score_pred_eval(company, investors, truth, score_function, threshold, g):
    result = Counter()
    neighbors = g.predecessors(company)
    
    for investor in investors:
        # Do no prediction on existing links
        if investor in g.neighbors:
            continue
            
        # Calculate the score_function score
        score = score_function(investor, company, g)

        # Make a prediction based on the threshold
        link_predicted = (score > threshold)

        # Record if it was a true/false pos or true/false neg
        if link_predicted:
            if (investor, company) in truth:
                result['tp'] += 1
            else:
                result['fp'] += 1
        else:
            if (investor, company) not in truth:
                result['tn'] += 1
            else:
                result['fn'] += 1
    
    return result

15514

In [59]:
# Test Company Score/Pred/Eval
random_truth = set(map(lambda x: (random.randint(1,2000), random.randint(2001, 4000)), range(1,10000)))
company_score_pred_eval(2001, range(1, 2000), random_truth, rand_score, 0.5, [])


Counter({'fn': 2, 'fp': 1020, 'tn': 976, 'tp': 1})

In [76]:
#### SINGLE THRESHOLD VERSION WITH MULTIPROCESSING
# It's inefficient to calculate and store all of the scores
# between all possible links
#
# truth is a set-like object containing all links (investor, company)
# created in the validation period
#
# score_function: a function that calculates a normalized
# score given investor, company, g
#
# threshold is a float above which values will be predicted positive
#
# g is the underlying graph
#
# RETURNS the confusion matrix.
def multi_eval_prec_recall(truth, score_function, threshold, g):
    # initialize multiprocessing pool
    pool = mp.Pool(processes=4)
        
    # initialize counters
    full_results = Counter({'tp':0, 'tn':0, 'fp':0, 'fn':0})
    
    # initialize list of companies and investors
    investors = map(lambda x: x[0],
                    filter(lambda x: x[1] == 0,
                    nx.get_node_attributes(g, "bipartite").items()))
    companies = map(lambda x: x[0],
                    filter(lambda x: x[1] == 1,
                    nx.get_node_attributes(g, "bipartite").items()))
    
    # Generate a function that is a function of just company, from MP
    evaluator = functools.partial(company_score_pred_eval, 
                                  truth=truth,
                                  score_function=score_function, 
                                  threshold=threshold,
                                  g=g,
                                  investors=investors)
    
    # iterate over all companies
    query = pool.imap(evaluator, companies, math.ceil(len(investors)/16))
    
    full_results = reduce(lambda x, y: x + y, query)

    pool.close()
    pool.join()

    return full_results

    

In [7]:
def calc_prec(confusion):
    (tp, tn, fp, fn) = (confusion["tp"], confusion["tn"], confusion["fp"], confusion["fn"])
    
    # calculate precision
    if (tp+fp) > 0:
        precision = tp/(tp+fp)
    else:
        precision = np.NaN
    
    return precision

def calc_recall(confusion):
    (tp, tn, fp, fn) = (confusion["tp"], confusion["tn"], confusion["fp"], confusion["fn"])
    
    # calculate recall
    if (tp+fn) > 0:
        recall = tp/(tp+fn)
    else:
        recall = np.NaN
    
    return recall

def calc_tpr(confusion):
    (tp, tn, fp, fn) = (confusion["tp"], confusion["tn"], confusion["fp"], confusion["fn"])
    
    # calculate tpr
    if (tp+fp) > 0:
        tpr = tp/(tp+fp)
    else:
        tpr = np.NaN
    
    return tpr

def calc_fpr(confusion):
    (tp, tn, fp, fn) = (confusion["tp"], confusion["tn"], confusion["fp"], confusion["fn"])
    
    # calculate tpr
    if (tp+fp) > 0:
        fpr = fp/(tn+fp)
    else:
        fpr = np.NaN
    
    return fpr

In [ ]:
multi_eval_prec_recall(,rand_score, 0.1, preG)

In [8]:
# Test Eval/Prec/Recall 
random_truth = set(map(lambda x: (random.randint(1,2000), random.randint(2001, 4000)), range(1,1000)))
%time multi_eval_prec_recall(random_truth, rand_score, 0.8, [])


CPU times: user 44 ms, sys: 16 ms, total: 60 ms
Wall time: 1.32 s


Counter({'fn': 785, 'fp': 798874, 'tn': 3196128, 'tp': 214})

In [9]:
def calc_multiple_results(truth, score_function, g, steps):
    print "START: ", datetime.datetime.now()
    results = []
    for threshold in map(lambda x: x/steps, range(1,steps)):
        results.append(multi_eval_prec_recall(truth, score_function, threshold, g))
        print threshold, datetime.datetime.now()

    print "END: ", datetime.datetime.now()
    
    return results

def calc_roc_data(results):
    tpr = [calc_tpr(result) for result in results]
    fpr = [calc_fpr(result) for result in results]
    
    return (tpr, fpr)


In [13]:
random_truth = set(map(lambda x: (random.randint(1,2000), random.randint(2001, 4000)), range(1,100000)))
results = calc_multiple_results(random_truth, rand_score, [], 20)
(tpr, fpr) = calc_roc_data(results)


START:  2015-12-05 20:58:22.437239
0.05 2015-12-05 20:58:24.882007
0.1 2015-12-05 20:58:27.413016
0.15 2015-12-05 20:58:30.005226
0.2 2015-12-05 20:58:32.585362
0.25 2015-12-05 20:58:35.139750
0.3 2015-12-05 20:58:37.707011
0.35 2015-12-05 20:58:40.280961
0.4 2015-12-05 20:58:43.332311
0.45 2015-12-05 20:58:45.954972
0.5 2015-12-05 20:58:48.785807
0.55 2015-12-05 20:58:52.106923
0.6 2015-12-05 20:58:55.712045
0.65 2015-12-05 20:58:59.416429
0.7 2015-12-05 20:59:02.748872
0.75 2015-12-05 20:59:05.995034
0.8 2015-12-05 20:59:09.512731
0.85 2015-12-05 20:59:12.033199
0.9 2015-12-05 20:59:14.431902
0.95 2015-12-05 20:59:16.768902
END:  2015-12-05 20:59:16.768971


In [19]:
plt.plot(fpr, tpr, label = "Random Prediction")

plt.title("ROC Curves")
plt.xlim([0,1])
#plt.ylim([0,1])
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.legend()
plt.show()

In [35]:
    # CONSIDER USE PRUNING TO REDUCE NUMBER OF COMPUTATIONS FOR CERTAIN TYPES OF COMPUTATIONS